In [1]:
import os
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, pipeline
import pandas as pd
import open_clip
import torch
from torchvision import transforms
from diffusers import DiffusionPipeline


/home/nenchevi/.conda/envs/setfit_pubmed/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
bitsandbytes library load error: Configured CUDA binary not found at /home/nenchevi/.conda/envs/setfit_pubmed/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda121.so
Traceback (most recent call last):
  File "/home/nenchevi/.conda/envs/setfit_pubmed/lib/python3.11/site-packages/bitsandbytes/cextension.py", line 313, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/home/nenchevi/.conda/envs/setfit_pubmed/lib/python3.11/site-packages/bitsandbytes/cextension.py", line 282, in get_native_library
    raise RuntimeError(f"Configured {BNB_BACKEND} binary not found at {cuda_binary_path}")
RuntimeError: Configured CUDA binary not found at /home/nenchevi/.conda/envs/setfit_pubmed

In [3]:
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe.to("cuda")

# if using torch < 2.0
# pipe.enable_xformers_memory_efficient_attention()


Loading pipeline components...: 100%|██████████| 7/7 [00:02<00:00,  2.86it/s]


StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.35.2",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [4]:
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k') # https://huggingface.co/laion/CLIP-ViT-B-32-laion2B-s34B-b79K
tokenizer = open_clip.get_tokenizer('ViT-B-32')

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [2]:
df = pd.read_parquet("tabelle.parquet")

In [3]:
df.head(2)

,filename,text,group,list_tokens,list_of_cunks
0,21AP15_Raw.txt,do I do creative things with language y er er...,scz,"[creative, things, language, trick, question, ...","[creative, things, language, trick, question, ..."
1,21AP15_Raw.txt,do I do creative things with language y er er...,scz,"[creative, things, language, trick, question, ...","[mean, language, like, f, sound, comes, mouth,..."


In [4]:
df.shape

(4556, 5)

In [9]:
df["text_chunk"] = df["list_of_cunks"].apply(lambda x: " ".join(x))

In [15]:
df.head(2)

,filename,text,group,list_tokens,list_of_cunks,text_chunk
0,21AP15_Raw.txt,do I do creative things with language y er er...,scz,"[creative, things, language, trick, question, ...","[creative, things, language, trick, question, ...",creative things language trick question creati...
1,21AP15_Raw.txt,do I do creative things with language y er er...,scz,"[creative, things, language, trick, question, ...","[mean, language, like, f, sound, comes, mouth,...",mean language like f sound comes mouth creativ...


In [19]:
df.iloc[1000,-1]

'obviously s like play football kinda like energetic person cos'

In [11]:
def get_clips_scores(text):
    
    text_tokens = tokenizer(text)
    text_tokens = text_tokens.to(device)
    text_features = model.encode_text(text_tokens)
    
    clips = []
    
    for i in range(1, 26):
        result = pipe(prompt=text)
        image = result.images[0]
        image_tensor = preprocess(image).unsqueeze(0)  # shape: [1, 3, 224, 224]
        image_tensor = image_tensor.to(device)

        with torch.no_grad():
            image_features = model.encode_image(image_tensor)
            text_features = model.encode_text(text_tokens)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)
            clip_score = (image_features @ text_features.T).item()

        clips.append(clip_score)

    return clips 
        

In [13]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
df["text_chunk_clips"] = df["text_chunk"].apply(get_clips_scores)
df.to_parquet("tabelle_2.parquet")

 24%|██▍       | 12/50 [00:01<00:04,  8.31it/s]